In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import datetime
import re
from datetime import datetime
import matplotlib.dates as mdates
from scipy import signal
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import os
from tkinter import Tk
from tkinter.filedialog import askdirectory
from pathlib import Path
import ctypes  # An included library with Python install.
import gc
def Mbox(title, text, style):
    return ctypes.windll.user32.MessageBoxW(0, text, title, style)

def find_nearest_index(arr, target):
    nearest_index = None
    nearest_distance = float('inf')

    for index, value in enumerate(arr):
        distance = abs(value - target)
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_index = index
            
    return nearest_index

def filt(t,xn): 
    tt = t.to_numpy()
    b, a = signal.butter(5, 0.05)
    zi = signal.lfilter_zi(b, a)
    z, _ = signal.lfilter(b, a, xn, zi=zi*xn[0])
    z2, _ = signal.lfilter(b, a, z, zi=zi*z[0])
    yyy = signal.filtfilt(b, a, xn)
    #plt.scatter(t,yyy)
    return yyy

def piks(A):
    A = np.asarray(A)
    A = A[100:len(A)-100]
    #plt.plot(A)
    indices = [np.argmax(A),0]
    maximos = [0,0]
    corta_antes = A[0:np.argmax(A)-round(0.1*len(A)/10)]
    corta_despues = A[np.argmax(A)+round(0.1*len(A)/10):len(A)]
    
    if max(corta_antes) > max(corta_despues):
        yhat_antes = savgol_filter(corta_antes, window_length = max([round(len(corta_antes)/1000),5]), polyorder = 4)
        #plt.plot(yhat_antes)
        peaksa,_ = find_peaks(yhat_antes, height=0.2*max(A), distance=0.1*len(A)/3)
        if peaksa.size > 0:
            indices = [np.argmax(A), np.where(A == max(corta_antes))[0][0]]
            maximos = [max(A),max(corta_antes)]
    
    if max(corta_antes) < max(corta_despues):
        yhat_despues = savgol_filter(corta_despues, window_length = max([round(len(corta_despues)/1000),5]), polyorder = 4)
        #plt.plot(yhat_despues)
        peaksd,_ = find_peaks(yhat_despues, height=0.2*max(A), distance=0.1*len(A)/3)
        if peaksd.size > 0:
            indices = [np.argmax(A), np.where(A == max(corta_despues))[0][0] + np.argmax(A)]
            maximos = [max(A), max(corta_despues)]
        
    #plt.scatter(indices,maximos)
    return maximos

def negpiks(A):
    maxneg = piks(-A)
    minimos = np.asarray(maxneg)*-1
    return  minimos

def amplitudes(x,y):
    amp = [0,0]
    
    try:
        picos = piks(y)
        row1 = np.where(y == picos[0])[0][0]
        row2 = np.where(y == picos[1])[0][0]
        valles = negpiks(y)
        vrow1 = np.where(y == valles[0])[0][0]
        vrow2 = np.where(y == valles[1])[0][0]
        amp = np.asarray([picos[0]-valles[0],picos[1]-valles[1]])
    except Exception:
        pass  # Simply skip if there's an error

    return amp

def figsav(SMA,passno,x,y,t,var,folder):
    vrow = [0,0]
    valle = 0
    
    if max(y)>y[0]:
        #i = np.argmax(np.asarray(y))

        plt.rcParams.update({'font.size': 20})
        fig, ax  = plt.subplots(figsize=(12,8), layout='constrained') 
        fig.suptitle(str(SMA) + ' - Pass No.' + str(passno), fontsize=18)
        ax.plot_date(x,y, color = 'g', markersize=0, linestyle='solid', linewidth=1)

        picos = piks(y)
        row = [np.where(y == picos[0])[0][0],np.where(y == picos[1])[0][0]]

        ss = 0.08 #annotation away from '>'

        for r in [0,1]:
            try:
                plt.scatter(x[row[r]-10],picos[r], marker = '>')
                ax.annotate(str(round(picos[r])), (x[row[r]],picos[r]), (x[round(row[r]-ss*row[1])],picos[r]+ss*picos[1]),)
            except Exception:
                plt.scatter(x[row[r]-11],picos[r], marker = '>')
                ax.annotate(str(round(picos[r])), (x[row[r]-1],picos[r]), (x[round(row[r]-ss*row[1]-1)],picos[r]+ss*picos[1]),)


        plt.ylabel(var)
        ax.yaxis.set_major_locator(MultipleLocator(max([1,round(picos[1]/50)*10])))
        ax.yaxis.set_major_formatter('{x:.0f}')
        ax.yaxis.set_minor_locator(MultipleLocator(5))

        if t == "p":
            plt.ylim(-0.1*max(picos),1.2*max(picos))
            aguasarriba = 1000
            aguasabajo = 1000
            valle = min(y[min(row):max(row)])
            vrow11 = np.where(y == valle)[0][0] 
            plt.ylim(min([-0.1*max(picos),1.2*valle]),1.2*max(picos))
            try:
                plt.scatter(x[vrow11],valle, marker = '>')
                ax.annotate(str(round(valle)), (x[vrow11],valle), 
                                (x[round(vrow11-ss*vrow11)],valle+ss*valle),)
            except Exception:
                plt.scatter(x[vrow11-1],valle, marker = '>')
                ax.annotate(str(round(valle)), (x[vrow11-1],valle), 
                                (x[round(vrow11-ss*vrow11)],valle+ss*valle),)   

        if t == "st":
            aguasarriba = 2000
            aguasabajo = len(x)-row[1]-100
            
            valle = min(y[min(row):max(row)])
            vrow11 = np.where(y == valle)[0][0] 
            plt.ylim(min([-0.1*max(picos),1.2*valle]),1.2*max(picos))

            found_valid_index = False
            attempt_index = vrow11

            #chatGPT
            while not found_valid_index and attempt_index >= 0:
                try:
                    # Attempt to access and plot the current index
                    plt.scatter(x[attempt_index], valle, marker='>')
                    ax.annotate(str(round(valle)), 
                                (x[attempt_index], valle), 
                                (x[round(attempt_index - ss * attempt_index)], valle + ss * valle))
                    found_valid_index = True  # Valid index found, exit the loop
                except IndexError:
                    # Decrement the index and try again
                    attempt_index -= 1

            if not found_valid_index:
                print("No valid index found.")
            ###
       
        if t == "sl":
            aguasarriba = 2000
            aguasabajo = 1000

            valles = negpiks(y)
            if valles[0] < -1:
                vrow1 = np.where(y == valles[0])[0][0]
                if valles[0] < -0.1:
                    vrow2 = np.where(y == valles[1])[0][0]
                    vrow = [vrow1,vrow2]  
            plt.ylim(min([-0.1*max(picos),1.2*min(valles)]),1.2*max(picos))
            
            #chatGPT
            for r in range(len(valles)):  # Assuming you're iterating over the indices of valles
                # Find a valid index for scatter
                found_valid_index = False
                attempt_index = vrow[r] - 10

                while not found_valid_index and attempt_index >= 0:
                    try:
                        plt.scatter(x[attempt_index], valles[r], marker='>')
                        found_valid_index = True  # Valid index found
                    except IndexError:
                        attempt_index -= 1

                if not found_valid_index:
                    plt.scatter(x[0], valles[r], marker='>')
                    print("HEY!!!!!!!!!!!!!!!!!!!!!!!!!! There is an annotation error for {} data for pass {}".format(t, passno))

                # Find a valid index for annotation
                found_valid_annotation_index = False
                attempt_annotation_index = vrow[r]

                while not found_valid_annotation_index and attempt_annotation_index >= 0:
                    try:
                        ax.annotate(str(round(valles[r])), 
                                    (x[attempt_annotation_index], valles[r]), 
                                    (x[round(attempt_annotation_index - ss * vrow[1])], valles[r] + ss * valles[1]))
                        found_valid_annotation_index = True  # Valid index found
                    except IndexError:
                        attempt_annotation_index -= 1

                if not found_valid_annotation_index:
                    ax.annotate(str(round(valles[r])), 
                                (x[attempt_annotation_index - 1], valles[r]), 
                                (x[round(attempt_annotation_index - ss * vrow[1])], valles[r] + ss * valles[1])) 
            ###

        plt.xlabel('Time on ' + str(x[0].date()))

        try:
            plt.xlim(x[row[0]-aguasarriba],x[row[1]+aguasabajo])
        except Exception:
            plt.xlim(x[500],x[row[1]+aguasabajo])


        
        locator = mdates.SecondLocator(interval=1)
        formatter = mdates.DateFormatter('%H:%M:%S')
        mlocator = mdates.MicrosecondLocator(interval=100000)
        mformatter = mdates.DateFormatter('')
        ax.xaxis.set_major_locator(locator)
        ax.xaxis.set_major_formatter(formatter)
        ax.xaxis.set_minor_locator(mlocator)
        ax.xaxis.set_minor_formatter(mformatter)
        ax.grid(which='both')                                                                                           
        ax.grid(which='minor', alpha=0.2)                                                
        ax.grid(which='major', alpha=0.7)

        figname = 'Pass No.' + str(passno) + ', ' + str(var)+'.png' # 'Pass No.' + str(passno) + ', ' +'Temperature'+'.png'
        full_path = os.path.join(folder, figname)
        plt.savefig(full_path)
        plt.close(fig)
        
        
    else:
        #Mbox('HEY!!!!!!!!!!!!!!!!!!!!!!!!!!', "There is no {} data for pass {}".format(t,passno), 1)
        print("HEY!!!!!!!!!!!!!!!!!!!!!!!!!! There is no {} data for pass {}".format(t,passno))
    return

In [2]:
SMA = 'D4-50-12.5-0 Air 120+120psi Load 10+10kip Speed 5mph Spacing 3.5ft'
from PyQt5.QtWidgets import QApplication, QFileDialog

# Create a QApplication instance
app = QApplication([])

# Open the file dialog
directory = 'C:/Users/jjg5/70001'
#QFileDialog.getExistingDirectory(None, "Select Input Folder")
directory2 = "//samba.campuscluster.illinois.edu/illinois-eng-centers-ict-shared/R27-216/Rutting Test Responses/70001"
#QFileDialog.getExistingDirectory(None, "Select Output Folder")

# Change to your desired directory
directory = Path(directory).resolve()


# Verify the change
print("Current Working Directory:", Path.cwd())

Current Working Directory: C:\Users\jjg5


In [3]:
if directory:
    
    # Get the list of filenames in the directory
    filenames = os.listdir(directory)
    
    # Use tqdm to create a progress bar for the loop
    for filename in tqdm(filenames, desc="Processing files", unit="file"):
        # Create the full file path
        file_path = os.path.join(directory, filename)
        
        # Check if it's a file and if the filename contains the word 'Pressure'
        if os.path.isfile(file_path) and "Pressure" in filename:
            #print(f'Matching file found: {filename}')
            #print(f'Found file: {file_path}')

            match = re.search(r'Pass(\d+)\.', filename)

            passno = match.group(1)

            file = filename.split('-')[0]
            
            try:
            
                S = pd.read_csv(file + '-Strain_Pass'+ str(passno) +'.csv', skiprows=4, usecols=['time', 'SF1', 'SF2'])
                S.time = pd.to_datetime(S.time, errors='coerce')
                S = S.dropna()
                S['SF1'] = S['SF1'].apply(pd.to_numeric, errors='coerce')
                S['SF2'] = S['SF2'].apply(pd.to_numeric, errors='coerce')
                S['SF1'] = S['SF1'] + np.ones(S.shape[0])* - -221.188
                S['SF2'] = S['SF2'] + np.ones(S.shape[0])* - -41.833
                if float(passno) == 1:
                    S = S.tail(7500)
                    S = S.reset_index(drop=True)
                S.loc[:,"S1f"] = filt(S.time,S.SF1)
                S.loc[:,"S2f"] = filt(S.time,S.SF2)

                    



                P = pd.read_csv(file + '-Pressure_Pass'+ str(passno) +'.csv', skiprows=4, usecols=['time', 'Pressure 4', 'Pressure 5'])
                P.time = pd.to_datetime(P.time, errors='coerce')
                P = P.dropna()
                P = P.rename(columns={'Pressure 4': 'p3','Pressure 5': 'p4'})
                P = P.astype({'p3':'float', 'p4':'float'}, errors='ignore')
                #to psi
                P['p4'] = P['p4']*145.038 + np.ones(P.shape[0])*+0.007*145.038
                P['p3'] = P['p3']*145.038 + np.ones(P.shape[0])*-0.03*145.038
                if float(passno) == 1:
                    P = P.tail(8000)
                    P = P.reset_index(drop=True)
                P.loc[:,"p1f"] = filt(P.time,P.p3)
                P.loc[:,"p2f"] = filt(P.time,P.p4)
                

                T = pd.read_csv(file + '-Temp_Pass'+ str(passno) +'.csv', skiprows=4, usecols=['time', 'TA4', 'TF1', 'TF3', 'TF4'])
                T.time = pd.to_datetime(T.time, errors='coerce')
                T = T.dropna()
                T = T.rename(columns={'TA4': 'Air'})
                T['Air'] = T['Air'].apply(pd.to_numeric, errors='coerce')
                T['TF1'] = T['TF1'].apply(pd.to_numeric, errors='coerce')
                #T['TF2'] = T['TF2'].apply(pd.to_numeric, errors='coerce')
                T['TF3'] = T['TF3'].apply(pd.to_numeric, errors='coerce')
                T['TF4'] = T['TF4'].apply(pd.to_numeric, errors='coerce')
                
                #######passno = int(passno) + 300   #############################################OJO!!!!!!!!!!!!!!!!!!!!!!!!
                
                if int(passno) % 1000 ==0:
                    figsav(SMA,passno,S.time,S.S1f,"sl",'Longitudinal microstrain',directory2)
                    figsav(SMA,passno,S.time,S.S2f,"st",'Transverse microstrain',directory2)
                    figsav(SMA,passno,P.time,P.p2f,"p",'Vertical Stress on Pressure Cell 1 (psi)',directory2)

                    figsav(SMA,passno,P.time,P.p1f,"p",'Vertical Stress on Pressure Cell 2 (psi)',directory2)

                    fig, ax = plt.subplots(1,1,sharex=True, sharey=True, figsize=(12,4), layout='constrained')
                    plt.plot_date(T.time,T.TF4*9/5+32, color = 'r', label='SMA', markersize=0, linestyle='solid', linewidth=1)
                    plt.plot_date(T.time,T.TF1*9/5+32, color = 'y', label='Bottom of Base', markersize=0, linestyle='solid', linewidth=1.5)
                        #plt.plot_date(T.time,T.TF2*9/5+32, color = 'g', label='Base', markersize=0, linestyle='solid', linewidth=0.5)
                    plt.plot_date(T.time,T.TF3*9/5+32, color = 'b', label='Bottom of SMA', markersize=0, linestyle='solid', linewidth=1)
                    plt.plot_date(T.time,T.Air*9/5+32, color = 'c', label='Air', markersize=0, linestyle='solid', linewidth=0.5)
                    #plt.title("{} on Pass No.".format(SMA) + str(passno))
                    plt.legend()
                    plt.grid()
                    plt.xlabel('Time on ' + str(T.time[1].date()))
                    fig.supylabel("Temperature ($^\circ$F) ")
                    figname = 'Pass No.' + str(passno) + ', ' +'Temperature'+'.png'
                    full_path = os.path.join(directory2, figname)
                    plt.savefig(full_path)
                    plt.close(fig)


                Tavg_1below = float('nan')

                peaks_first_pressure_cell = np.array([np.nan, np.nan])
                mode__first_pressure_cell = float('nan')
                min_p1 = float('nan')
     
                peaks_second_pressure_cell = np.array([np.nan, np.nan])
                mode__s_pressure_cell = float('nan')
                min_p2 = float('nan')

                peaks_first_strain_gauge = np.array([np.nan, np.nan])
                mins_first_strain_gauge = np.array([np.nan, np.nan])
                mode__l_strain = float('nan')
                amplitudel = np.array([np.nan, np.nan])

                peaks_second_strain_gauge = np.array([np.nan, np.nan])
                mins_second_strain_gauge = float('nan')
                mode__t_strain = float('nan')
                amplitudet = np.array([np.nan, np.nan])

                
                
                
                #Data Extraction
                Tavg_1below = np.asarray(np.mean(T.TF4))*9/5+32
                if max(P.p1f)>P.p1f[0]:
                    peaks_first_pressure_cell = piks(P.p1f)
                    mode__first_pressure_cell = P.p1f[500:1000].round().mode().iloc[0]
                    y = P.p1f
                    picos = peaks_first_pressure_cell
                    row = [np.where(y == picos[0])[0][0],np.where(y == picos[1])[0][0]]
                    min_p1 = min(y[min(row):max(row)])
                if max(P.p2f)>P.p2f[0]:
                    peaks_second_pressure_cell = piks(P.p2f)
                    mode__s_pressure_cell = P.p2f[500:1000].round().mode().iloc[0]
                    y = P.p2f
                    picos = peaks_second_pressure_cell
                    row = [np.where(y == picos[0])[0][0],np.where(y == picos[1])[0][0]]
                    min_p2 = min(y[min(row):max(row)])
                if max(S.S1f)>S.S1f[0]:
                    peaks_first_strain_gauge = piks(S.S1f)
                    mins_first_strain_gauge = negpiks(S.S1f)
                    mode__l_strain = S.S1f[200:300].round().mode().iloc[0]
                    amplitudel = amplitudes(S.time,S.S1f)
                if max(S.S2f)>S.S2f[0]:
                    peaks_second_strain_gauge = piks(S.S2f)
                    y = S.S2f
                    picos = peaks_second_strain_gauge
                    row = [np.where(y == picos[0])[0][0],np.where(y == picos[1])[0][0]]
                    mins_second_strain_gauge = min(y[min(row):max(row)])
                    mode__t_strain = S.S2f[500:1000].round().mode().iloc[0]
                    amplitudet = [peaks_second_strain_gauge[0]-mode__t_strain,peaks_second_strain_gauge[1]-mins_second_strain_gauge]

                extracted = np.concatenate([passno, 
                                            peaks_first_pressure_cell, mode__first_pressure_cell, peaks_first_pressure_cell - mode__first_pressure_cell, 
                                            min_p1,
                                            peaks_second_pressure_cell, mode__s_pressure_cell, peaks_second_pressure_cell - mode__s_pressure_cell,
                                            min_p2,
                                            peaks_first_strain_gauge, mins_first_strain_gauge, amplitudel, mode__l_strain,
                                            peaks_first_strain_gauge - mode__l_strain, mins_first_strain_gauge - mode__l_strain,
                                            peaks_second_strain_gauge, mins_second_strain_gauge, amplitudet, mode__t_strain,
                                            peaks_second_strain_gauge - mode__t_strain, mins_second_strain_gauge - mode__t_strain,
                                            Tavg_1below], axis=None)
                df2 = pd.DataFrame(extracted).T
                df2.columns=['pass',
                             'pressure1peak2','pressure1peak1','pressure1base','pressure1peak1rel','pressure1peak2rel', 'valleyp1',
                             'pressure2peak2', 'pressure2peak1', 'pressure2base','pressure2peak1rel', 'pressure2peak2rel', 'valleyp2',
                             'strainLpeak1', 'strainLpeak2','strainLmin1', 'strainLmin2', 'LAmp1', 'LAmp2', 'strain1base',
                             'strainLpeak1rel', 'strainLpeak2rel','strainLmin1rel', 'strainLmin2rel',
                             'strainTpeak1', 'strainTpeak2','strainTvalley', 'TAmp1', 'TAmp2', 'strain2base',
                             'strainTpeak1rel', 'strainTpeak2rel','strainTmin1rel',
                             'testT']

                #if passno == str(1):
                #csvfile = file + '_Pass'+ str(passno) +'.csv'
                #full_path = os.path.join(directory2, csvfile)
                #df2.to_csv(full_path, index=False)

                A = pd.read_csv('appended.csv')
                A = pd.concat([A,df2])
                A.to_csv('appended.csv', index=False)
                del S, P, T, A  # Remove DataFrames after use
                gc.collect()  # Trigger garbage collection
            except Exception:
                #Mbox('HEY!!!!!!!!!!!!!!!!!!!!!!!!!!', 'There was an error with pass number: " + str(passno)', 1)
                print("HEY!!!!!!!!!!!!!!!!!!!!!!!!!!   There was an error with pass number: " + str(passno))
else:
    print("No folder selected.")

Selected directory: C:/Users/jjg5/70001
HEY!!!!!!!!!!!!!!!!!!!!!!!!!!   There was an error with pass number: 70631
HEY!!!!!!!!!!!!!!!!!!!!!!!!!!   There was an error with pass number: 70643
HEY!!!!!!!!!!!!!!!!!!!!!!!!!!   There was an error with pass number: 70658
HEY!!!!!!!!!!!!!!!!!!!!!!!!!!   There was an error with pass number: 70661
HEY!!!!!!!!!!!!!!!!!!!!!!!!!!   There was an error with pass number: 70670
HEY!!!!!!!!!!!!!!!!!!!!!!!!!!   There was an error with pass number: 70686
HEY!!!!!!!!!!!!!!!!!!!!!!!!!!   There was an error with pass number: 70693
HEY!!!!!!!!!!!!!!!!!!!!!!!!!!   There was an error with pass number: 70707
HEY!!!!!!!!!!!!!!!!!!!!!!!!!!   There was an error with pass number: 70732
HEY!!!!!!!!!!!!!!!!!!!!!!!!!!   There was an error with pass number: 70741
HEY!!!!!!!!!!!!!!!!!!!!!!!!!!   There was an error with pass number: 70747
HEY!!!!!!!!!!!!!!!!!!!!!!!!!!   There was an error with pass number: 70748
HEY!!!!!!!!!!!!!!!!!!!!!!!!!!   There was an error with pass

In [4]:
A = pd.read_csv('appended.csv')
full_path = os.path.join(directory2, 'appended.csv')
A.to_csv(full_path, index=False)
A = A.drop(A.index)
A.to_csv('appended.csv', index=False)